# 3. Regresión Logística

Supongamos que tenemos un conjunto de datos $\mathcal{D} = \{(x_i, y_i) \in \mathbb{R}^p \times \{0,1\}: i \in[m]\}$. El método de *regresión logística* asume que $Pr[y_i|x_i,\beta] \sim Bernoulli(\mu_i)$. Queremos encontrar el modelo $\hat{\beta} \in \mathbb{R}^p$ que mejor se ajuste a $\mathcal{D}$. Si encontramos $\hat{\beta}$ podemos usarlo para modelar $Pr[y|\boldsymbol{x},\hat{\beta}]$ y predecir la etiqueta $\hat{y} \in \{0,1\}$ de un nueva dato $\boldsymbol{x}$.

La *log-verosimilitud negativa* está dada por
$$ F(\beta) := LVN(\beta) = - \sum_{i=1}^m [y_i \log \mu_i + (1-y_i) \log (1-\mu_i)]$$

1. Escriba las expresiones para $\nabla F(\beta)$ y $\nabla^2 F(\beta)$.

$$\nabla F(\beta) = X^T(\mu-y)$$
$$\nabla^2 F(\beta) = X^TSX$$

2. Use estas expresiones para implementar `grad_F` y `hess_F` en Python. Ambas funciones tienen como argumento `(X,y,beta)` con `X.shape=(m,p)`, `y.shape=(m,)` y `beta.shape=(p,)` 

Primero definimos la función *sigmoide* que se usará en ambas fuciones, así como las bibliotecas que se usarán

In [15]:
import numpy as np
import pandas as pd
from sklearn import preprocessing

def sigm(X):
  return 1/(1+np.exp(-X))

Posteriormente definimos el gradiente

In [16]:
def grad_F(X,y,beta):
  m = sigm(X@beta) 
  return X.T@(m-y)

De igual manera, definimos el hessiano

In [17]:
def hess_F(X,y,beta): 
  m = sigm(X@beta) 
  S = np.outer(m.T,(1-m)) 
  return (X.T@S)@X

3. Implemente el método de *máximo descenso* para minimizar $F(\beta)$. Elija un $\beta^0$ aleatorio y una tolerancia de $\epsilon = 10^{-8}$. Encuentre $a^k$ usando la *condición de Armijo* (vista en clase). Usar $\tau = 0.5$ y $\gamma = 0.1$.

Primero definimos la función de *log-verosimilitud negativa*.

In [18]:
def LVN(X,y,beta):
  m = sigm(X@beta) 
  return -(y@np.log(m) + (1-y)@np.log(1-m))

A continuacion se define la fución del método de *máximo descenso* usandoo *Armijo*.

In [19]:
def grad_desc_armijo(X,y):
  
  #Definimos los parámetros
  m,p = X.shape
  beta = np.zeros(p)# np.random.uniform(size = p)
  tau = 0.5
  gamma = 0.1
  epsilon = 10**-8 
  alfa = 0.00001 
  niter = 0
  
  #Estos son los valores iniciales
  p = -grad_F(X,y,beta)
  leftf = LVN(X,y,beta + alfa*p)
  rightf = LVN(X,y,beta) + gamma*alfa*(grad_F(X,y,beta)@p)
  
  #Iteraciones hasta que se compla la condición
  while leftf + epsilon < rightf:
    niter += 1
    alfa = alfa*tau
    beta = beta + alfa*p
    p = -grad_F(X,y,beta)
    leftf = LVN(X,y,beta + alfa*p)
    rightf = LVN(X,y,beta) + gamma*alfa*(grad_F(X,y,beta)@p)
  
  return beta, niter 

4. Implemente un clasificador de regresión logística para obtener $\hat{y}$. En Python `yhat = pred(x, betahat)`.

La función de predicción es

In [20]:
def pred(x, betahat):
  return((sigm(x@betahat)>0.5)*1)

5. Implemente la siguiente función de datos.

In [21]:
def datos(modo='entrena'):
  gid = 'd932a3cf4d6bdeef36a7230fff959301'
  tail = '64b604aedff376b7757b533d1c93685ce19b2077/bcdata'
  url = 'https://gist.githubusercontent.com/rodrgo/%s/raw/%s' % (gid, tail)
  df = pd.read_csv(url, sep=',')
  df = df.drop(columns=['Unnamed: 32', 'id'])
  var = 'diagnosis'
  df.loc[df[var] == 'M', [var]] = 1
  df.loc[df[var] == 'B', [var]] = 0
  X_cols = [c for c in df.columns if not c is var]
  X, y = df[X_cols].to_numpy(), df[var].to_numpy()
  idx = np.random.permutation(X.shape[0])
  train_idx, test_idx = idx[:69], idx[69:]
  idx = train_idx if modo == 'entrena' else test_idx
  return X[idx,:], y[idx]


Cree dos conjuntos de datos:<br>
`X_entrena, y_entrena = datos()`<br>
`X_prueba, y_prueba = datos(modo='prueba')`<br>
Use `(X_entrena, y_entrena)` para ajusar un valor $\hat{\beta}$(`betahat`) usando máximo descenso. Use la función `pred`
para predecir la etiqueta de cada punto en el conjunto de datos `X_prueba`. Compare esta respuesta con `y_prueba` para obtener el error de predicción de su modelo.

Primero creamos el conjunto de datos

In [22]:
X_entrena, y_entrena = datos()
X_prueba, y_prueba = datos(modo='prueba')

A continuación usaremos haremos la predicción de los datos con las funciones anteriores. Primero se normalizan los datos

In [23]:
scaler = preprocessing.MinMaxScaler()

Xst_entrena = scaler.fit_transform(X_entrena)
Xst_prueba = scaler.transform(X_prueba)

Ahora entrenamos el modelo

In [24]:
betahat_armijo, i = grad_desc_armijo(Xst_entrena,y_entrena)

Con $\hat{\beta}$ haremos la predicción

In [25]:
yhat = pred(Xst_prueba, betahat_armijo)

A continuación calculamos el error de predicción

In [26]:
np.mean(yhat-y_prueba)


0.628

6. Ahora implemente en Python el método de Newton para minimizar $F(\beta)$.

In [27]:
def grad_desc_newton(X,y):
  
  #Definimos los parámetros
  m,p = X.shape
  beta = np.random.uniform(size = p)
  tau = 0.5
  gamma = 0.1
  epsilon = 10**-8 
  alfa = 0.000000000000001 
  niter = 0
  
  #Estos son los valores iniciales
  p = -np.linalg.inv(hess_F(X,y,beta))@grad_F(X,y,beta)
  leftf = LVN(X,y,beta + alfa*p)
  rightf = LVN(X,y,beta) + gamma*alfa*(grad_F(X,y,beta)@p)
  
  #Iteraciones hasta que se compla la condición
  while leftf + epsilon < rightf:
    niter += 1
    alfa = alfa*tau
    beta = beta + alfa*p
    p = -grad_F(X,y,beta)
    leftf = LVN(X,y,beta + alfa*p)
    rightf = LVN(X,y,beta) + gamma*alfa*(grad_F(X,y,beta)@p)
    print(LVN(X,y,beta))
  
  return beta, niter